# J1–J2–J3 Exchange Frustration: Single Simulation & S(q) Analysis

This notebook demonstrates a single UppASD simulation with J1–J2–J3 exchange (frustrated) and focuses on momentum-space analysis via the static structure factor S(q).

## Scientific objective
- Run a single finite-temperature simulation with controlled J1/J2/J3
- Compute and visualize the static structure factor S(q) from `sq._UppASD_.out`

## Learning goals
- Build and run the model with `SpinSystem` and `ExchangeShellTable`
- Produce and analyze `sq._UppASD_.out` using the included plotting helpers
markdown
markdown
## Real-space snapshot and S(q)

Below we provide two complementary visualizations of the final magnetic state:
- A real-space heatmap of the out-of-plane magnetization `mz` (useful to spot domains/skyrmions),
- The static structure factor `|S(q)|` in momentum space (peaks indicate ordering wavevectors).

Run the real-space cell first (it uses `results.restart` and `results.coord`), then run the `S(q)` cell to compare features between real and reciprocal space.
code
python

- (Optional) Inspect time-series of magnetization and energy as a tangential example

## Recommended execution order
Run cells from top to bottom. Re-run downstream cells after modifying setup parameters.

In [ ]:
# Imports and basic setup (restored from reference notebook)
import importlib.util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Install uppasd if missing in the kernel (uncomment to enable)
# if importlib.util.find_spec("uppasd") is None:
#     print("Installing uppasd…")
#     !pip install --pre --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple uppasd

# Core UppASD classes for model setup, execution, and results parsing
from uppasd.core.system import SpinSystem
from uppasd.core.exchange import ExchangeShellTable, DMIShellTable
from uppasd.input.inputdata import ASDInput
from uppasd.run.simulator import ASDWorkspace, UppASDSimulator
from uppasd.core.results import ASDResults
from uppasd.viz import plot_sq, plot_heatmap
from uppasd.viz import results_snapshot_grid
import numpy as np

# Temperature-token helpers (useful for switching initial-phase syntax)
from uppasd import set_temperature_token, validate_temperature_token

print("✅ Imports restored. Ensure you run cells that define `system`, `exchange`, and `inp` before preparing the workspace.")

## Step 1 — Define the crystal and initial magnetic state

We start from a minimal model to keep the physics and code path transparent.

### What each object means
- `cell`: lattice vectors that define periodic geometry
- `positions`: atom coordinates in the unit cell
- `species`: integer species labels (important for multi-component systems)
- `moments`: initial spin vectors per atom

Here we initialize moments along `+z`, which gives a simple reference state for observing relaxation behavior.

In [ ]:
# Set up a single unit cell with one atom
a = 1.0
cell = a * np.eye(3,3)

positions = np.array([[0.0 , 0.0, 0.0]])
natom = positions.shape[0]
species = np.ones(natom, dtype=int)

moments = np.zeros((natom, 3))
moments[:, 2] = 1.0

system = SpinSystem(cell, positions, species, moments)

## Step 2 — Define exchange interactions

The `ExchangeShellTable` encodes Heisenberg interactions shell-by-shell.

In the next code cell we define a simple frustrated J1–J2–J3 example and add bonds to the exchange table.

In [ ]:
exchange = ExchangeShellTable()

# Definition of exchange interactions
J1 = -1.0
J2 = J1/2.0
J3 = J1

# Exchange couplings J_ij in mRy (shell-resolved)
J_ij = [J1, J2, J3]
r_ij = np.array([[a, 0, 0], 
                 [a, a, 0], 
                 [2*a, a, 0]
                ])

for ij, jij in enumerate(J_ij):
    if abs(jij)>0.0:
        exchange.add_bond(1, 1, ij+1, r_ij[ij], jij)

## Step 3 — Configure input blocks

`ASDInput` organizes UppASD keywords by conceptual blocks (`system`, `initial`, `simulation`, `measurables`).

We use an LLG initial phase (`ip_mode='S'`) with scalar syntax:
- `ip_temp`: initial-phase temperature
- `ip_nstep`: number of initial-phase steps
- `ip_timestep`, `ip_damping`: integration controls

Then we set production simulation controls and request output needed for time-series analysis (`averages` and `totenergy`).

### Note: 
In this particular example we set the initial phase parameters to not thermalize the system properly so we can track the relaxation procedure during the measurement phase. For production simulations, the thermalization needs to be properly done.

**Import & setup:** An imports/setup cell has been added above — run it first. If you have previously defined `system`, `exchange`, and `inp`, the workspace preparation cell will initialize the run directory.

In [ ]:
# Define simulation and measurement parameters (ASDInput)
inp = ASDInput()
Nx = 64
Ny = 64
Nz = 1

# Define a temperature to ensure same value is given in initial and measurement phases (optional)
Temperature = 0
Temperature_shift = 1.0

inp.block("system").set(
    ncell=(Nx, Ny, Nz),
    bc=("P", "P", "0"),
    do_prnstruct=2,
    sym=2,
 )

# Initial phase (LLG) using scalar syntax
inp.block("initial").set(
    ip_mode="H",
    ip_temp=Temperature+Temperature_shift,
    ip_mcnstep=10000,
    initmag=1,
 )

# Main simulation settings
inp.block("simulation").set(
    mode="S",
    nstep=5000,
    timestep=1e-15,
    damping=0.5,
    temperature=Temperature,
 )

# Request energy/averages/cumulant output
inp.block("measurables").set(plotenergy=1, do_avrg="Y", do_cumu="Y", do_sc="C", sc_sep='100', qpoints="C")

## Step 4 - Prepare workspace and run UppASD

The next two code cells do the full execution workflow:
1. Create a clean run directory and write all required input/interaction files
2. Initialize UppASD, run initial phase, perform measurement, and finalize

Keeping these steps explicit is useful for debugging and for teaching what happens under the hood.

### Note:
The `sim.initialize()` and `sim.finalize()` steps are always needed for any simulation.

In [ ]:
# Prepare workspace (will only run if system/inp/exchange are defined)
workdir = "J1J2J3_run"
if 'system' in globals() and 'inp' in globals() and 'exchange' in globals():
    ws = ASDWorkspace(workdir, clean=True)
    ws.prepare(system=system, inp=inp, exchange=exchange)
    print(f'Workspace prepared at {workdir}')
else:
    print("Variables 'system', 'inp', or 'exchange' not defined. Define them before preparing workspace.")

In [ ]:
sim = UppASDSimulator(ws)
sim.initialize()
sim.run_init_phase()
sim.measure()
sim.finalize()

In [ ]:
# Post-processing for a single run: magnetization and energy vs time
results = ASDResults(workdir)
print("Available tables:", results.available)

In [ ]:
# Plot a snapshot from `moment` using viz helpers
if results.restart is None:
    print('No moment table found (moment.<simid>.out missing)')
else:
    iter_id, grid = results_snapshot_grid(results, key='restart', component=2, idx=-1)
    plot_heatmap(grid, title=f'm_z snapshot iter={iter_id}', cmap='RdBu_r', vmin=-1.0, vmax=1.0)

In [ ]:
# Plot the static structure factor S(q) from results.sq
# Requires results.sq to be present (read from sq.<simid>.out)
if getattr(results, 'sq', None) is None:
    print('No S(q) table found (sq.<simid>.out missing)')
else:
    plot_sq(results.sq, fftshift=False, cmap='magma', title='|S(q)|')